## Import & Constants

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from koapy import KiwoomOpenApiPlusEntrypoint
from koapy import KiwoomOpenApiPlusTrInfo
from datetime import datetime
from playsound import playsound
import time
pd.set_option('display.max_rows', None)

In [2]:
DAEBI_SIGN = {1: '상한', 2: '상승', 3: '보합', 4: '하한', 5: '하락'}
ADJUSTED_SIGN = {1:'유상증자', 2:'무상증자', 4:'배당락', 8:'액면분할', 16:'액면병합', 32:'기업합병'
                 , 64:'감자', 256:'권리락'}

## Log in

In [3]:
entrypoint = KiwoomOpenApiPlusEntrypoint()
entrypoint.EnsureConnected()
print(entrypoint.GetConnectState())

2024-03-27 07:29:38,903 [DEBUG] Testing if client is ready... - KiwoomOpenApiPlusEntrypoint.py:45
2024-03-27 07:29:48,922 [DEBUG] Client is not ready - KiwoomOpenApiPlusEntrypoint.py:47
2024-03-27 07:29:48,925 [DEBUG] Creating a new server... - KiwoomOpenApiPlusEntrypoint.py:48


1


## Functional Function

In [4]:
def endbeep():
    print("playing sound")
    for _ in range(2):
        playsound("sound/alert1.wav")

In [5]:
def IsConnected():
    print(entrypoint.GetConnectState())

## Code

In [6]:
CODE_LIST = list(pd.read_csv("DB/20240314_stocklist.csv")['0'])
NAME_LIST = [entrypoint.GetMasterCodeName(code) for code in CODE_LIST]
cl_bef = entrypoint.GetCodeListByMarketAsList("0")

In [7]:
# print(entrypoint.IsConditionLoaded())
# print(entrypoint.EnsureConditionLoaded())
# print(entrypoint.IsConditionLoaded())

# # cl = entrypoint.GetConditionNameListAsList()
# cml = entrypoint.GetCodeListByCondition('All_Stock', with_info=False)
# cml = cml[:2559] # cml.index('453860')
# pd.Series(cml).to_csv("DB/20240314_stocklist.csv", index=False)

In [8]:
def TR_info(trcode:str, prt:bool=False):
    trinfo = KiwoomOpenApiPlusTrInfo.get_trinfo_by_code(trcode)
    output = {}
    output["Name"] = trinfo.inputs_name
    output["Input"] = list(x.name for x in trinfo.inputs)
    output["Single"] = list(x.name for x in trinfo.single_outputs)
    output["Multi"] = list(x.name for x in trinfo.multi_outputs)
    if prt: print(output)
    return output

In [9]:
def call_TR(trcode:str, param:list, screenno=None, end_date=None, debug=False):
    output = {'single': {}, 'multi': []}
    trinfo = TR_info(trcode)
    rqname = trinfo['Name']
    inputs = {k:v for k, v in zip(trinfo['Input'], param)}
    
    if debug: print(f'Requesting data for request name: {rqname}, trcode: {trcode}')
    
    for event in entrypoint.TransactionCall(rqname, trcode, screenno, inputs):
        if debug: print("Got request")
        
        single_names = event.single_data.names
        single_values = event.single_data.values
        output['single'] = {k:v for k, v in zip(single_names, single_values)}
        
        multi_records = [v.values for v in event.multi_data.values]
        multi_df_partial = pd.DataFrame.from_records(multi_records, columns=trinfo['Multi'])
        output['multi'].append(multi_df_partial)
        if end_date != None:
            oldest = datetime.strptime(multi_df_partial.iloc[-1]['일자'], '%Y%m%d')
            if oldest < end_date:
                break
    
    output['multi'] = pd.concat(output['multi'], axis=0).reset_index(drop=True)
    return output

In [10]:
def save_data(func, st, ed, errlim):
    i = 0; err_cnt = 0
    while st+i <= ed:
        try:
            for one in func(st+i, ed):
                i += one
                err_cnt = 0
        except Exception as e: 
            print(f"Error on {st+i}, rep={err_cnt+1}")
            err_cnt += 1
            if err_cnt >= errlim:
                print(f"error count > {errlim}, terminate process.")
                print(e.args)
                i += 99999
            else: time.sleep(180)

In [11]:
def opt10059_saveall(st, en):
    '''
    2006-01-03부터 데이터 존재.
    '''
    pathdir = 'DB/Investor'
    start_iter = st; end_iter = en; iter_cnt = 0  # start_iter, end_iter번째 데이터 포함
    for code, name in zip(CODE_LIST, NAME_LIST):
        iter_cnt += 1
        if iter_cnt < start_iter: continue
        if iter_cnt > end_iter: break
            
        print(f"Operating for: {code} {name}, {iter_cnt}")
        tr_example = call_TR('opt10059', ['20240313', code, '2', '0', '1']
                             , end_date=datetime(2006,1,3))
        df = tr_example['multi']
        df['일자'] = pd.to_datetime(df['일자'])
        df.set_index('일자', inplace=True)
        df = df[['개인투자자', '외국인투자자', '기관계', '금융투자', '보험', '투신', '기타금융'
                 , '은행', '연기금등', '사모펀드', '기타법인']]
        df = df.rename(columns={'개인투자자': '개인', '외국인투자자': '외국인', '기관계': '기관'})
        df = df.loc[:datetime(2006,1,3)]
        df.to_csv(f"{pathdir}/{code}.csv", index=True)
        print(f"Completed: {code} {name}")
        yield 1
        

In [12]:
def opt10081_saveall(st, ed):
    pathdir = 'DB/Chart'
    start_iter = st; end_iter = ed; iter_cnt = 0  # start_iter, end_iter번째 데이터 포함
    
    for code, name in zip(CODE_LIST, NAME_LIST):
        iter_cnt += 1
        if iter_cnt < start_iter: continue
        if iter_cnt > end_iter: break
            
        print(f"Operating for: {code} {name}, {iter_cnt}")
        for i in (0, 1):
            addpath = 'Not_Adjusted' if i else 'Adjusted'
#             tr = call_TR('opt10081', [code, '20240313', i]
#                                  , end_date=datetime(2006,1,3))
            df = entrypoint.GetDailyStockDataAsDataFrame(code, start_date="20240313"
                            , end_date="20060103", include_end=True
                            , adjusted_price=i)
            df['일자'] = pd.to_datetime(df['일자'])
            df.set_index('일자', inplace=True)
            df = df[['시가', '고가', '저가', '현재가', '거래량', '거래대금', '수정주가구분'
                 , '수정비율']]
            df = df.loc[:datetime(2006,1,3)]
            df = df.rename(columns={'현재가': '종가'})
            df.to_csv(f"{pathdir}/{addpath}/{code}.csv", index=True)
        print(f"Completed: {code} {name}")
        yield 1

In [15]:
%%time
save_data(opt10081_saveall, 1200, 2559, 2)
endbeep()

Operating for: 065710 서호전기, 1200


2024-03-27 07:31:57,623 [DEBUG] Received 600 records from 2024-03-13 00:00:00 to 2021-10-07 00:00:00 for code 065710 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:31:58,135 [DEBUG] Received 600 records from 2021-10-06 00:00:00 to 2019-05-03 00:00:00 for code 065710 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:31:58,670 [DEBUG] Received 600 records from 2019-05-02 00:00:00 to 2016-11-17 00:00:00 for code 065710 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:31:59,159 [DEBUG] Received 600 records from 2016-11-16 00:00:00 to 2014-06-12 00:00:00 for code 065710 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:32:15,142 [DEBUG] Received 600 records from 2014-06-11 00:00:00 to 2012-01-05 00:00:00 for code 065710 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:32:15,630 [DEBUG] Received 600 records from 2012-01-04 00:00:00 to 2009-08-13 00:00:00 for code 065710 - KiwoomOpenApiPlusServiceClientStubWrapper

Completed: 065710 서호전기
Operating for: 065770 CS, 1201


2024-03-27 07:32:52,398 [DEBUG] Received 600 records from 2024-03-13 00:00:00 to 2021-10-07 00:00:00 for code 065770 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:32:52,882 [DEBUG] Received 600 records from 2021-10-06 00:00:00 to 2019-05-03 00:00:00 for code 065770 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:32:53,351 [DEBUG] Received 600 records from 2019-05-02 00:00:00 to 2016-11-17 00:00:00 for code 065770 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:33:09,351 [DEBUG] Received 600 records from 2016-11-16 00:00:00 to 2014-06-12 00:00:00 for code 065770 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:33:09,853 [DEBUG] Received 600 records from 2014-06-11 00:00:00 to 2012-01-05 00:00:00 for code 065770 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 07:33:10,395 [DEBUG] Received 600 records from 2012-01-04 00:00:00 to 2009-08-13 00:00:00 for code 065770 - KiwoomOpenApiPlusServiceClientStubWrapper

Completed: 065770 CS
Operating for: 065950 웰크론, 1202


...

Completed: 452400 이닉스
Operating for: 452430 사피엔반도체, 2558


2024-03-27 18:37:43,741 [DEBUG] Received 17 records from 2024-03-13 00:00:00 to 2024-02-19 00:00:00 for code 452430 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 18:37:44,415 [DEBUG] Received 17 records from 2024-03-13 00:00:00 to 2024-02-19 00:00:00 for code 452430 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718


Completed: 452430 사피엔반도체
Operating for: 453340 현대그린푸드, 2559


2024-03-27 18:38:00,476 [DEBUG] Received 227 records from 2024-03-13 00:00:00 to 2023-04-10 00:00:00 for code 453340 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718
2024-03-27 18:38:00,919 [DEBUG] Received 227 records from 2024-03-13 00:00:00 to 2023-04-10 00:00:00 for code 453340 - KiwoomOpenApiPlusServiceClientStubWrapper.py:718


Completed: 453340 현대그린푸드
playing sound
CPU times: total: 1min 54s
Wall time: 11h 6min 15s


In [14]:
# len(CODE_LIST)  # = 2559